In [48]:
import pandas as pd
import sqlite3
import os
import sys
import matplotlib as mpl
mpl.use('TkAgg')
mpl.rcParams['font.sans-serif'] = ['FangSong']
import matplotlib.pyplot as plt

In [45]:
os.chdir('/Users/kunyue/project_personal/my_project/gaokao_back/gaokao/processor')
cnx = sqlite3.connect('../../db.sqlite3')
df_sch_major = pd.read_sql_query("SELECT * FROM gaokao_schoolmajor", cnx)
school = pd.read_sql_query("SELECT * FROM gaokao_school", cnx)[['sch_id', 'sch_tags', 'sch_name']]
schoolRank = pd.read_sql_query("SELECT * FROM gaokao_gaokaometarank", cnx)

In [30]:
def get_lucky_sch(wenli,batch_name):
    df_trait = df_sch_major[df_sch_major['wenli']==wenli]
    df_trait = df_trait[df_trait['batch_name']==batch_name]
    df_trait['min_score_diff'] = df_trait['min_score_diff'].map(int)
    df_trait['min_score_rank'] = df_trait['min_score_rank'].map(int)
    df_trait = df_trait[(df_trait['min_score_diff']>0) & (df_trait['min_score_diff']<700)]
    df_trait = df_trait[(df_trait['min_score_rank']>0) & (df_trait['min_score_rank']<1000000)]
    df_detail = df_trait[['school_id','academic_year','enroll_major_name','min_score_diff','min_score_rank']]
    df_grp_min = df_trait.groupby(['school_id','academic_year']).agg(min_score_diff=('min_score_diff',min),min_score_rank=('min_score_rank',max)).reset_index()
    df_grp_min = pd.merge(df_grp_min,df_detail,on=['school_id','academic_year','min_score_diff','min_score_rank'])
    df_min_last_year = df_grp_min[df_grp_min['academic_year']<'2019'].groupby(['school_id','academic_year','min_score_diff','min_score_rank']).head(1)
    df_min_last_year.rename(columns={'min_score_diff':'last_min_score','min_score_rank':'last_min_rank'},inplace=True)
    df_min_next_year = df_grp_min[df_grp_min['academic_year']>='2017']
    df_min_next_year['academic_year'] = df_min_next_year['academic_year'].map(lambda x: str(int(x)-1))
    df_min_next_year.rename(columns={'min_score_diff':'next_min_score','min_score_rank':'next_min_rank'},inplace=True)
    df_result = pd.merge(df_min_last_year,df_min_next_year,on=['school_id','academic_year'])
    df_result['diff_score'] = df_result.apply(lambda x:(int(x['last_min_score']) - int(x['next_min_score']))/int(x['last_min_score']),axis=1)
    df_result['diff_rank'] = df_result.apply(lambda x:(int(x['last_min_rank']) - int(x['next_min_rank']))/int(x['last_min_rank']),axis=1)
    df_result = df_result.sort_values(by=['diff_score'])
    return df_result

In [33]:
def show_result(df):
    school['211'] = school['sch_tags'].map(lambda x: '211' if str(x).find('211') >= 0 else '')
    school['985'] = school['sch_tags'].map(lambda x: '985' if str(x).find('985') >= 0 else '')
    result = pd.merge(school,df,left_on='sch_id', right_on='school_id')
    result = result.drop(['sch_tags','sch_id'],axis=1)
    return result


In [23]:
def get_school_rule(wenli, batch_name):
    '''
    录取人数变多，对低分排名近3年逐年升高，即录取分数越来越高
    :return:
    '''
    # 三年上涨幅度每年超过5%
    df_trait = df_sch_major[df_sch_major['wenli'] == wenli]
    df_trait = df_trait[df_trait['batch_name'] == batch_name]
    df_trait['min_score_diff'] = df_trait['min_score_diff'].map(int)
    df_trait['min_score_rank'] = df_trait['min_score_rank'].map(int)
    df_grp_min = df_trait.groupby(['school_id','academic_year']).agg(min_score_rank=('min_score_rank',max)).reset_index()
    df_grp_min['diff'] = df_grp_min.sort_values(by=['school_id','academic_year']).groupby(['school_id'])['min_score_rank'].diff()
    df_grp_min_2019_lt = df_grp_min[df_grp_min['academic_year']<'2019']
    df_grp_min_2019_lt = df_grp_min_2019_lt.dropna(subset=['diff'])
    df_grp_min_2019_lt['diff_percent'] = df_grp_min_2019_lt['diff']/df_grp_min_2019_lt['min_score_rank']
    df_grp_min_2019_lt_count = df_grp_min_2019_lt[df_grp_min_2019_lt['diff_percent']<-0.05].groupby(['school_id'])['diff_percent'].count().reset_index()
    df_grp_min_2019_lt_count_2 = df_grp_min_2019_lt_count[df_grp_min_2019_lt_count['diff_percent']>=2]
    return df_grp_min_2019_lt_count_2

In [ ]:
def plot_rank(wenli,academic_year):
    schoolT = schoolRank[schoolRank['academic_year']==academic_year]
    schoolT = schoolT[schoolT['wenli']==wenli]
    schoolT = schoolT.sort_values(by = ['score'])
    schoolT = schoolT[schoolT['score']>=200]
    schoolT['range'] = pd.cut(schoolT['score'],100)
    school_grp = schoolT.groupby(['range'])['rank'].sum().reset_index()
    array_x = school_grp['range']
    array_y = school_grp['rank']
    print(array_x)


    fig = plt.figure()
    # 生成第一个子图在1行2列第一列位置
    ax1 = fig.add_subplot(121)
    # 生成第二子图在1行2列第二列位置
    # ax2 = fig.add_subplot(122)
    # 绘图并设置柱子宽度0.5
    ax1.bar(array_x, array_y, width=0.1)
    # 绘图默认柱子宽度0.8
    # plt.show()


In [ ]:
plot_rank('2','2019')


In [25]:
result_rule = get_school_rule('2','本科第一批')


In [34]:
result_lucky = get_lucky_sch('2','本科第一批')
result_lucky = show_result(result_lucky)

/anaconda3/envs/py3.6/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/Users/kunyue/python_env_tf11/lib/python3.6/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [35]:
result_analyse = pd.merge(result_lucky,result_rule,on=['school_id'])
print(result_analyse)

             sch_name  211  985                 school_id academic_year  \
0              中国人民大学  211  985  52ac2e99747aec013fcf4f0b          2016   
1              中国人民大学  211  985  52ac2e99747aec013fcf4f0b          2016   
2              中国人民大学  211  985  52ac2e99747aec013fcf4f0b          2016   
3              中国人民大学  211  985  52ac2e99747aec013fcf4f0b          2018   
4              中国人民大学  211  985  52ac2e99747aec013fcf4f0b          2018   
..                ...  ...  ...                       ...           ...   
525      电子科技大学(沙河校区)            570c7212df6537f0c9a6f0f9          2018   
526  中国石油大学(北京)克拉玛依校区            58b5434c7cadef952da48fb8          2016   
527  中国石油大学(北京)克拉玛依校区            58b5434c7cadef952da48fb8          2017   
528  中国石油大学(北京)克拉玛依校区            58b5434c7cadef952da48fb8          2017   
529  中国石油大学(北京)克拉玛依校区            58b5434c7cadef952da48fb8          2018   

     last_min_score  last_min_rank                   enroll_major_name_x  \
0               164    